In [ ]:
import os    
import findspark
import re
from functools import reduce
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

os.environ['SPARK_HOME'] = 'C:\\Users\\msi\\Desktop\\spark\\spark-3.0.1-bin-hadoop3.2'
findspark.init()
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler, VectorSlicer, PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC, NaiveBayes, DecisionTreeClassifier
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col, lit, udf, mean as _mean, isnan, sum as _sum, log as _log
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import FloatType, ArrayType, IntegerType
from pyspark.sql import DataFrame
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import Evaluator

In [ ]:
train_df= spark.read.csv('C:\\Users\\msi\\Onedrive\\MOA\\train_features.csv', header=True, inferSchema=True)
target_df = spark.read.csv('C:\\Users\\msi\\Onedrive\\MOA\\train_targets_scored.csv', header=True, inferSchema=True)
test_df = spark.read.csv('C:\\Users\\msi\\Onedrive\\MOA\\test_features.csv', header=True, inferSchema=True)

In [ ]:
train_df.cache()
target_df.cache()
test_df.cache()

In [ ]:
def encode_cat_features(df, cat_features):

  indexed_cols = [''.join([col_name, '_indexed']) for col_name in cat_features]
  encoded_cols = [''.join([col_name, '_encoded']) for col_name in cat_features]
  string_indexers = [StringIndexer(inputCol=cat_features[i], outputCol=indexed_cols[i]) for i in range(len(cat_features))]
    
  encoder = OneHotEncoder(inputCols=indexed_cols, outputCols=encoded_cols)
  
  pipline = Pipeline(stages=string_indexers + [encoder])
  
  encoded_df = pipline.fit(df).transform(df)
  encoded_df = encoded_df.drop(*indexed_cols + cat_features)

  return encoded_df

def normalize_features(df, cols, normalizer, output_cols, if_drop=True):
  """
  """
  normalizer_lst = []
  vectorized_cols = []
  vector_assembers = []
  
  if isinstance(cols, list):
    cols = {'cols': cols}
  
  if isinstance(output_cols, str):
    output_cols = {'cols': output_cols}
  
  for k, v in cols.items():
    
    temp_normalizer = normalizer.copy()
    vectorized_col = ''.join([output_cols[k], '_v'])
    vector_assember = VectorAssembler(inputCols=v, outputCol=vectorized_col)
    
    temp_normalizer.setInputCol(vectorized_col)
    temp_normalizer.setOutputCol(output_cols[k])
    
    normalizer_lst.append(temp_normalizer)
    vectorized_cols.append(vectorized_col)
    vector_assembers.append(vector_assember)
  
  pipline = Pipeline(stages=vector_assembers + normalizer_lst)
  normalized_df = pipline.fit(df).transform(df).drop(*vectorized_cols)
  
  if if_drop:
    
    for k, v in cols.items():
      
      normalized_df = normalized_df.drop(v)
  
  return normalized_df

def add_pca_features(df, g_cols, c_cols, k=40):
  
  ## normalize g-col and c-col
  std_scaler = StandardScaler(withMean=True)
  
  input_cols = {
    'g_cols': g_cols, 
    'c_cols': c_cols}
  
  output_cols = {
    'g_cols': 'g_normalized', 
    'c_cols': 'c_normalized'}
  
  normalized_df = normalize_features(df, input_cols, std_scaler, output_cols, if_drop=False)
  
  ## perform PCA on g-cols and c-cols
  g_col_pca = PCA(k=k, inputCol='g_normalized', outputCol='g_col_pca')
  c_col_pca = PCA(k=k, inputCol='c_normalized', outputCol='c_col_pca')
  
  pipeline = Pipeline(stages=[g_col_pca, c_col_pca])
  pca_df = pipeline.fit(normalized_df).transform(normalized_df)
  
  return pca_df
  
def add_stats_features(df, g_cols, c_cols):
  
  @udf('double')
  def cols_sum(*lst):

    return sum(lst)

  @udf('double')
  def cols_mean(*lst):

    n = len(lst)
    s = sum(lst)

    return s / n

  @udf('double')
  def cols_var(*lst):

    n = len(lst)
    s = sum(lst) / n
    total = 0

    for x in lst:

      total += (x - s)**2 

    return total / n
  
  @udf('double')
  def cols_min(*lst):
    
    return min(lst)
  
  @udf('double')
  def cols_max(*lst):
    
    return max(lst)
  
  stats_dict = {
    'min_stats': cols_min,
    'max_stats': cols_max,
    'var_stats': cols_var,
    'mean_stats': cols_mean,
    'sum_stats': cols_sum
  }
  
  for name, func in stats_dict.items():
    
    df = df.withColumn(''.join(['g_cols_', name]), func(*[col(g_col) for g_col in g_cols]))
    df = df.withColumn(''.join(['c_cols_', name]), func(*[col(c_col) for c_col in c_cols]))
  
  return df

def add_kmeans_features(df, g_cols, c_cols, k=2, num_iter=10):
  
  kmeans_g = KMeans(k=k, featuresCol=g_cols, predictionCol='g_col_k_mean', seed=16)
  kmeans_c = KMeans(k=k, featuresCol=c_cols, predictionCol='c_col_k_mean', seed=16)
  
  kmeans_df = kmeans_g.fit(df).transform(df)
  kmeans_df = kmeans_c.fit(kmeans_df).transform(kmeans_df)
  
  return kmeans_df

def feature_engineering(df, num_cluster=2, num_comp=40, num_iter=10):
  
  ## get g-col and c-col
  g_cols = list(filter(lambda v: re.match('g-.+', v), df.columns))
  c_cols = list(filter(lambda v: re.match('c-.+', v), df.columns))
  
  ## PCA
  pca_df = add_pca_features(df, g_cols, c_cols, num_comp)

  ## stats features on g and c cols
  stats_df = add_stats_features(pca_df, g_cols, c_cols)
  
  ## add k-means features
  kmeans_df = add_kmeans_features(stats_df, g_cols='g_normalized', c_cols='c_normalized', k=num_cluster, num_iter=num_iter)
  
  return kmeans_df

def get_correlation(df, threshold=0.95, feature_col='all_features', plot=False):
    
    from pyspark.ml.stat import Correlation
    from pyspark.sql.types import FloatType, ArrayType, IntegerType
    import numpy as np
    
    r1 = Correlation.corr(df, feature_col)
    r1 = r1.select(f'pearson({feature_col})').collect()[0][f'pearson({feature_col})'].toArray().tolist()
    
    if plot:
        plt.figure(figsize=(20, 10))
        cor_plt_array = np.array(r1)
        sns.heatmap(cor_plt_array)
        
    r1 = spark.createDataFrame(r1)
    gv = sc.broadcast(threshold)
    va = VectorAssembler(inputCols=r1.columns, outputCol='features')
    r1 = va.transform(r1)
    
    def find_element(row, gv):
    
        lst = []

        for i in range(len(row)):

            if (row[i] >= gv.value) | (row[i] <= -gv.value):
                lst.append(i)
        
        if len(lst) == 1:
            lst = []
            
        return lst

    m = udf(lambda x: find_element(x, gv), ArrayType(IntegerType()))
    
    return r1.withColumn('new_col', m('features')).select('new_col')
    

In [ ]:
## add indicator column to both train and test so we can combine them later
train_df = train_df.withColumn('is_test', lit(0))
test_df = test_df.withColumn('is_test', lit(1))

## Combine train and test df
full_df = train_df.union(test_df)

In [ ]:
## encode features
target_cols = ['cp_type', 'cp_dose']
encoded_df = encode_cat_features(full_df, target_cols)

In [ ]:

## feature engineering
fe_df = feature_engineering(encoded_df, num_comp=20, num_iter=5)

## select all the feature columns

pca_cols = list(filter(lambda v: re.match('.+_pca', v), fe_df.columns))
stats_cols = list(filter(lambda v: re.match('.+_stats', v), fe_df.columns))
k_means_cols = list(filter(lambda v: re.match('.+_k_mean', v), fe_df.columns))
cat_cols = list(filter(lambda v: re.match('.+_encoded', v), fe_df.columns)) + ['cp_time']
all_cols = pca_cols + stats_cols + k_means_cols + cat_cols
## stack them to a single feature vector
vector_assember_train = VectorAssembler(inputCols=all_cols, outputCol='all_features')
fe_df = vector_assember_train.transform(fe_df)

## Check their correlation
corr_df = get_correlation(fe_df, 0.96, plot=True)
corr_df.dropDuplicates().show()

In [ ]:
## 40 PCA columns
## looking for position 3, 6, 7, 8
(stats_cols + k_means_cols + cat_cols)[3]

In [ ]:
(stats_cols + k_means_cols + cat_cols)[6]

In [ ]:
(stats_cols + k_means_cols + cat_cols)[7]

In [ ]:
(stats_cols + k_means_cols + cat_cols)[8]

In [ ]:
high_corr_features = ['c_cols_max_stats', 'g_cols_mean_stats', 'c_cols_mean_stats', 'g_cols_sum_stats']

In [ ]:
## stack them to a single feature vector again, and remove those features with high correlation
vector_assember_train = VectorAssembler(inputCols=[col for col in all_cols if col not in high_corr_features], outputCol='all_features')
fe_df = vector_assember_train.transform(fe_df.drop('all_features'))

In [ ]:
## normalize all the features
normalizer = StandardScaler(withMean=True)
cols = ['all_features']
output_cols = 'features'
fe_df = normalize_features(fe_df, cols, normalizer, output_cols, if_drop=False)

## split train, test df
fe_train = fe_df.filter(fe_df['is_test'] == 0)
final_test = fe_df.filter(fe_df['is_test'] == 1).select(['sig_id', 'features'])

## join training target with training features
labels = target_df.drop('sig_id').columns
final_train = fe_train.join(target_df, ['sig_id']).select(*(['sig_id','features'] + labels))


In [ ]:
## train test split
(train, validation) = final_train.randomSplit([0.8, 0.2], 16)

train.cache()
validation.cache()
train_df.unpersist()
test_df.unpersist()
target_df.unpersist()
fe_train.unpersist()
fe_df.unpersist()
final_train.unpersist()
encoded_df.unpersist()
corr_df.unpersist()

In [ ]:
from tqdm import tqdm
# Multilabel Classifier
class MultiLabelClassifier:
    
    def __init__(self, clf, labels, feature_col,  
                 hyperparameters={}, 
                 predict_col=['probability','prediction'],
                 method=lambda prob_col, pred_col: float(pred_col if len(prob_col) == 1 else prob_col[1])):
        '''
        Initialize a multilabelclassifier
        clf: the model to use
        labels: a list of labels to predict
        feature_col: the feature column
        predict_col: the prediction column where the prediction is located
        hyperparameters: all optional hyperparameters that can tune
        method: a method of how to get the final prediction for one class
        '''
        self.clf = clf
        self.labels = labels
        self.feature_col = feature_col
        self.predict_col = predict_col
        self.hyperparameters = hyperparameters
        self.method = method
        self._trained_clfs = []

    def fit(self, train):
        train.cache()
        self._trained_clfs = [self.clf(labelCol=label, featuresCol=self.feature_col, **self.hyperparameters)
                              .fit(train) 
                              for label in tqdm(self.labels)]
        train.unpersist()
        return self

    def transform(self, x_test):
        # convert method to udf
        get_predict = udf(self.method,FloatType())
        #target assembler
        va = VectorAssembler(inputCols=self.labels, outputCol='targets')
        ## transform this vector self.output_col to an array
        select_cols = [self.feature_col, 'targets', 'sig_id']
        res = va.transform(x_test).select(*select_cols)
        for i, clf in tqdm(enumerate(self._trained_clfs)):
            res = clf.transform(res)
            new_col = self.labels[i]
            res = res.withColumn(new_col, get_predict(*self.predict_col))
            select_cols.append(new_col)
            res = res.select(*select_cols)
        self.res = res
        return res.select(*select_cols[2:])
    
    def score(self):
        #target assembler 
        va = VectorAssembler(inputCols=self.labels, outputCol='predicts')
        ## transform this vector self.output_col to an array
        df = va.transform(self.res).select('targets', 'predicts')
        df = df.withColumn('targets', vector_to_array('targets'))
        df = df.withColumn('predicts', vector_to_array('predicts'))
        import math
        @udf('double')
        def log_loss(y, y_hat):
            r = 0
            cut = 1e-15
            for t, p in zip(y, y_hat):
                p = max(min(p, 1-cut),cut)
                r += t * math.log(p) + (1 - t) * math.log(1 - p)
            return r/len(y)
        df = df.select(log_loss('targets','predicts').alias('log_loss'))
        return df.select((-_mean(col('log_loss'))).alias('score'))
    
    def param_search_cv(self, train, grid_map, fold_num):
                   
        train.cache()
        best_params = []
        
        def _extract_best_params(cv_model):
        
            best_param_dict = {}
            scores = cv_model.avgMetrics
            best_param = cv_model.getEstimatorParamMaps()[scores.index(min(scores))]

            for k, v in best_param.items():
                best_param_dict[k.name] = v

            return best_param_dict
            
        for label in tqdm(self.labels):
            
            clf = self.clf(labelCol=label, featuresCol=self.feature_col)
            evaluator = MultilabelEvaluator(method=self.method, predictionCol=self.predict_col, labelCol=label)
            cv = CrossValidator(estimator=clf, estimatorParamMaps=grid_map, numFolds=fold_num, evaluator=evaluator, parallelism=3)
            cv_model = cv.fit(train)
            best_model = cv_model.bestModel
            best_param = _extract_best_params(cv_model)

            self._trained_clfs.append(best_model)
            best_params.append(best_param) 
        
        return best_params
    

class MultilabelEvaluator(Evaluator):
    
    def __init__(self, method, predictionCol=['probability', 'prediction'], labelCol="label"):
        
        self.predictionCol = predictionCol
        self.labelCol = labelCol
        self.method = method
        
    def _evaluate(self, dataset):
        
        get_predict = udf(self.method,FloatType())
        dataset = dataset.withColumn('pred_prob', get_predict(*self.predictionCol))
        cut = 1e-15
        new_dataset = dataset.select((-col(self.labelCol) * _log(col('pred_prob') + cut) - (1.0 - col(self.labelCol)) * _log(1.0 - col('pred_prob') + cut)).alias('log_loss'))        
        score = new_dataset.select(_mean(col('log_loss')).alias('score')).collect()[0]['score']
        return score
        
    def isLargerBetter(self):
        return False
    

In [ ]:
def convert_to_array(prob_col, pred_col):
            ## solve na problem, if len(prob_col) == 1, we use the prediction col
            converted_prob_col = prob_col.toArray().tolist()
            
            if len(converted_prob_col) == 1:
                
                return pred_col
            
            else:
                
                return converted_prob_col[1]

In [ ]:
## test
clf = DecisionTreeClassifier
grid_map = ParamGridBuilder().addGrid(clf().maxDepth, [2, 5]).addGrid(clf().impurity, ['gini', 'entropy']).build()
method = convert_to_array
dt = MultiLabelClassifier(clf, labels, 'features')

In [ ]:
best_params = dt.param_search_cv(train, grid_map, 2)

In [ ]:
# import time
# print('Start calculating')
# sample_id = "sig_id"
# start = time.time()
# y = target_df.select(*[sample_id] + labels)
# score(y, res1, sample_id).show()
# print('Calculation finished with time:', time.time() - start)

In [ ]:
# def cross_validation(df, estimator, k=2):
    
#     num_per_fold = 100 // k * 0.01
#     num = [num_per_fold] * (k - 1) + [1 - num_per_fold * (k - 1)] 
#     k_fold_df = df.randomSplit(num)
#     acc = []
    
#     def unionAll(*dfs):
#         return reduce(DataFrame.unionAll, dfs)
    
#     for i, fold in enumerate(k_fold_df):
        
#         validation = fold
#         train = unionAll(*[k_fold_df[j] for j in range(len(k_fold_df)) if i != j])
        
#         clf = estimator.fit(train)
#         res = clf.transform(validation)
#         score = clf.score().collect()[0]['score']
        
#         validation.unpersist()
#         train.unpersist()
#         estimator.reset()
        
#         acc.append(score)
        
#     mean_acc =  sum(acc) / k
    
#     print(f'the acc are {acc}, the mean acc is {mean_acc}')
    
#     return mean_acc

# def extract_params(grid_map):
    
#     output_lst = []
    
#     for params in grid_map:
#         temp_dict = {}
        
#         for k, v in params.items():
#             temp_dict[k.name] = v
        
#         output_lst.append(temp_dict)
        
#     return output_lst

# def grid_search(train, estimator, grid_map, k=2):
    
#     params = extract_params(grid_map)
#     results = {}
    
#     for param in params:
#         print(f'evaluating param {param}')
        
#         estimator.hyperparameters = param
#         result = cross_validation(train, estimator, k)
        
#         if result in results.keys():
#             results[result].append(param)
            
#         else:
#             results[result] = [param]

#     return param

